# Наука за данни в облака: Методът "Azure ML SDK"

## Въведение

В този ноутбук ще научим как да използваме Azure ML SDK за обучение, разгръщане и използване на модел чрез Azure ML.

Предварителни изисквания:
1. Създадено е работно пространство в Azure ML.
2. Заредили сте [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) в Azure ML.
3. Качили сте този ноутбук в Azure ML Studio.

Следващите стъпки са:

1. Създаване на експеримент в съществуващо работно пространство.
2. Създаване на изчислителен клъстер.
3. Зареждане на набора от данни.
4. Конфигуриране на AutoML с помощта на AutoMLConfig.
5. Стартиране на AutoML експеримента.
6. Изследване на резултатите и избор на най-добрия модел.
7. Регистриране на най-добрия модел.
8. Разгръщане на най-добрия модел.
9. Използване на крайна точка.

## Специфични импорти за Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Инициализиране на работно пространство
Инициализирайте обект на работно пространство от запазена конфигурация. Уверете се, че конфигурационният файл присъства в .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Създаване на експеримент в Azure ML

Нека създадем експеримент с име 'aml-experiment' в работното пространство, което току-що инициализирахме.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Създаване на изчислителен клъстер
Ще трябва да създадете [изчислителна цел](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) за вашето AutoML изпълнение.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Данни
Уверете се, че сте качили набора от данни в Azure ML и че ключът има същото име като набора от данни.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Конфигурация на AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Автоматизирано машинно обучение


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Разгърнете най-добрия модел

Изпълнете следния код, за да разгърнете най-добрия модел. Можете да видите състоянието на разгръщането в портала на Azure ML. Тази стъпка може да отнеме няколко минути.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Използване на крайна точка
Можете да добавите входни данни към следния пример за вход.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
